This notebook aims to process each one of the splitted mp4 frames that are used to train the Martinez et al NN and see if MediaPipe is capable to detect correctly the 2D keypoints besides the drawbacks of Dustmask, Bodysuit and people in the background. 

The objective is to process each one of the frames and then delete manually the ones that are not detected propely. Then, the remaining frames will be used in a final analysis of the final solution to check if the proposed pipeline (MediaPipe + Martinez) is accurate enough to replace Kinect 

### Imports

In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
from glob import glob
from scipy.spatial import distance
import shutil

### Functions

In [2]:
mapping_numbers_variables = {0:'left_shoulder',
                             1:'right_shoulder',
                             2:'left_elbow',
                             3:'right_elbow',
                             4:'left_wrist',
                             5:'right_wrist'}

def split_2d_coordinates_array(coordinates_array):
    m = 0
    r = 2
    frame_coordinates = []
    for j in range(0,int(len(coordinates_array)/2)):
        frame_coordinates.append(coordinates_array[m:r])
        m = r
        r = r+2
    return(np.array(frame_coordinates))

def mediapipe_inference(frame):
    results = pose.process(frame)
    frame_list1 = []
    start_index = 0
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            #If landmark visibility is not 0.5 save a NaN and this value will be replaced by the previous recorded value
            #Reference & first value is the callibraition picture 
            
            if landmark.visibility>0.5:
                if start_index in indices_landmark_interest:
                    image_hight, image_width, _ = frame.shape
                    frame = cv2.circle(frame, (int(landmark.x*image_width), int(landmark.y*image_hight)), radius=5, color = (0,255,0), thickness=-1)   
                    frame_list1.append(landmark.x*image_width)
                    frame_list1.append(landmark.y*image_hight) 
                    start_index = start_index+1
                else:
                    start_index = start_index+1
            else: 
                start_index = start_index+1
    return(frame_list1,frame)

In [3]:
indices_landmark_interest = [11, 12, 13, 14, 15 ,16]
number_of_keypoints_to_detect = len(indices_landmark_interest)

#names_kinect = ['Xavi','Adri','Mireia']
names_kinect = ['Xavi','Adri']
numbers = ['01','02','03','04','05']
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(model_complexity=0,min_detection_confidence=0.8)

In [4]:
for name in names_kinect:
    for number in numbers:
        filenames = glob('../data/Webcam/'+str(name)+'/'+str(name)+str(number)+'/mp4_splitted/*.jpg')
        df = pd.DataFrame(columns=np.arange(0,(number_of_keypoints_to_detect*2)+1))
        for filename in filenames:
            image = cv2.imread(filename)
            timecode = float(filename.split('/')[-1].split('_')[-1].split('.jpg')[0])
            
#             #Defining the blurr filter intensity
#             blurred_img = cv2.GaussianBlur(image, (201, 201), 0)
#             #Generate a black image with the same shape as the image that we want to blurr
#             mask = np.zeros((1080, 1920, 3), dtype=np.uint8)
#             #Define bottom polytgon by its corners
#             contours1 = np.array([[0,1080],[0,820],[580,670],[1300,670],[1920,820],[1920,1080]])
#             #Draw the previous defined polygon in mask and fill it with white
#             mask = cv2.fillPoly(mask, pts = [contours1], color =(255,255,255))
#             contours2 = np.array([[580,100],[1300,100],[1300,670],[580,670]])
#             mask = cv2.fillPoly(mask, pts = [contours2], color =(255,255,255))
#             #Just keep white points normal and blurr black points
#             image = np.where(mask==np.array([255, 255, 255]), image, blurred_img)
    
            detected_keypoints, frame = mediapipe_inference(image)
            detected_keypoints.insert(0,timecode)
            df = df.append(pd.DataFrame(np.array(detected_keypoints).reshape(1,-1)))
        
            


            cv2.imwrite('../data/Webcam/'+str(name)+'/'+str(name)+str(number)+'/mp4_splitted_MPlabeled/'+filename.split('\\')[-1],frame)
        df.to_csv('../data/Webcam/'+str(name)+'/'+str(name)+str(number)+'/mp4_splitted_MPlabeled/'+str(name)+'_'+str(number)+'_mp_output_all.csv',header=None,index=None,sep=',')
                      